# Requirements

In [1]:
!git clone https://github.com/IlyaGusev/purano

Cloning into 'purano'...
remote: Enumerating objects: 1026, done.
remote: Counting objects: 100% (290/290), done.
remote: Compressing objects: 100% (205/205), done.
remote: Total 1026 (delta 147), reused 201 (delta 81), pack-reused 736
Receiving objects: 100% (1026/1026), 1.25 MiB | 18.54 MiB/s, done.
Resolving deltas: 100% (557/557), done.


In [2]:
%cd purano

/content/purano


In [3]:
!pip install -r requirements.txt
!pip install catboost
!pip install tensorflow-text tensorflow-hub
!pip install --upgrade transformers

     |████████████████████████████████| 20.1MB 58.4MB/s 
     |████████████████████████████████| 972kB 57.0MB/s 
     |████████████████████████████████| 215kB 58.3MB/s 
     |████████████████████████████████| 849kB 50.1MB/s 
     |████████████████████████████████| 122kB 59.4MB/s 
     |████████████████████████████████| 1.3MB 58.1MB/s 
     |████████████████████████████████| 266kB 57.0MB/s 
     |████████████████████████████████| 14.3MB 30.6MB/s 
     |████████████████████████████████| 71kB 10.9MB/s 
     |████████████████████████████████| 51kB 8.6MB/s 
     |████████████████████████████████| 61kB 10.0MB/s 
     |████████████████████████████████| 92kB 11.2MB/s 
     |████████████████████████████████| 61kB 10.6MB/s 
     |████████████████████████████████| 163kB 59.5MB/s 
     |████████████████████████████████| 184kB 58.3MB/s 
     |████████████████████████████████| 276kB 57.8MB/s 
     |████████████████████████████████| 112kB 58.5MB/s 
     |████████████████████████████████| 2.9MB 54.6MB

     |████████████████████████████████| 67.3MB 151kB/s 
     |████████████████████████████████| 3.4MB 4.6MB/s 
     |████████████████████████████████| 2.1MB 4.5MB/s 
     |████████████████████████████████| 3.3MB 27.2MB/s 
  Found existing installation: tokenizers 0.9.2
    Uninstalling tokenizers-0.9.2:
      Successfully uninstalled tokenizers-0.9.2
  Found existing installation: transformers 3.4.0
    Uninstalling transformers-3.4.0:
      Successfully uninstalled transformers-3.4.0


In [ ]:
!mkdir models/lang_detect
!mkdir models/cat_detect
!wget https://www.dropbox.com/s/hoapmnvqlknmu6v/lang_detect_v10.ftz -O models/lang_detect/lang_detect_v10.ftz
!wget https://www.dropbox.com/s/23x35wuet280eh6/ru_cat_v5.ftz -O models/cat_detect/ru_cat_v5.ftz

--2021-02-26 11:47:43--  https://www.dropbox.com/s/hoapmnvqlknmu6v/lang_detect_v10.ftz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6027:18::a27d:4812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/hoapmnvqlknmu6v/lang_detect_v10.ftz [following]
--2021-02-26 11:47:44--  https://www.dropbox.com/s/raw/hoapmnvqlknmu6v/lang_detect_v10.ftz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uced2387b1877d6e6e97411a3fec.dl.dropboxusercontent.com/cd/0/inline/BJrU0i41UwvZRtwDGZhCwyvS9jT17hdYanZD27Zai8R_Ky_PsyXTmkg4MoBlUaD6eoYafTR3u0sS9vnW0jQwW8yVyvzDhlZa6YYn6uQIe6Vyykui5OJFhDurVq9fzhchqEQ/file# [following]
--2021-02-26 11:47:44--  https://uced2387b1877d6e6e97411a3fec.dl.dropboxusercontent.com/cd/0/inline/BJrU0i41UwvZRtwDGZhCwyvS9jT17hdYanZD27Zai8R_Ky_PsyXTmkg4MoBlUaD6eoYafTR3u0sS9vnW0jQwW8y

# Data loading

In [ ]:
# May 25 (training)
!rm -rf data
!mkdir -p data
!cd data && wget https://data-static.usercontent.dev/DataClusteringDataset0525.tar.gz -O - | tar -xz && cd ../
# May 27 and 29 (test)
!cd data && wget https://data-static.usercontent.dev/DataClusteringDataset0527.tar.gz -O - | tar -xz && cd ../
!cd data && wget https://data-static.usercontent.dev/DataClusteringDataset0529.tar.gz -O - | tar -xz && cd ../

--2021-02-26 11:47:53--  https://data-static.usercontent.dev/DataClusteringDataset0525.tar.gz
Resolving data-static.usercontent.dev (data-static.usercontent.dev)... 178.128.255.27
Connecting to data-static.usercontent.dev (data-static.usercontent.dev)|178.128.255.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 145541247 (139M) [application/octet-stream]
Saving to: ‘STDOUT’

-                   100%[===================>] 138.80M  20.3MB/s    in 6.9s    

2021-02-26 11:48:00 (20.1 MB/s) - written to stdout [145541247/145541247]

--2021-02-26 11:48:00--  https://data-static.usercontent.dev/DataClusteringDataset0527.tar.gz
Resolving data-static.usercontent.dev (data-static.usercontent.dev)... 178.128.255.27
Connecting to data-static.usercontent.dev (data-static.usercontent.dev)|178.128.255.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 166313405 (159M) [application/octet-stream]
Saving to: ‘STDOUT’

-                   100%[=======

In [ ]:
%%writefile configs/cleaner.jsonnet
{
    "lang_detect_model_path": "models/lang_detect/lang_detect_v10.ftz",
    "cat_detect_model_path": "models/cat_detect/ru_cat_v5.ftz",
    "is_lower": true,
    "is_russian_only": true,
    "is_news_only": true
}

Overwriting configs/cleaner.jsonnet


In [ ]:
!rm -f 0525_parsed.db
!python3 -m purano.run_parse --inputs "data/20200525" --output-file 0525_parsed.db --fmt html --cleaner-config configs/cleaner.jsonnet
!rm -f 0527_parsed.db
!python3 -m purano.run_parse --inputs "data/20200527" --output-file 0527_parsed.db --fmt html --cleaner-config configs/cleaner.jsonnet
!rm -f 0529_parsed.db
!python3 -m purano.run_parse --inputs "data/20200529" --output-file 0529_parsed.db --fmt html --cleaner-config configs/cleaner.jsonnet

Parsed 10000 documents
Parsed 20000 documents
Parsed 30000 documents
Parsed 40000 documents
Parsed 50000 documents
Parsed 60000 documents
Parsed 70000 documents
Parsed 80000 documents
19380 will be saved
<class 'pandas.core.frame.DataFrame'>
Int64Index: 19380 entries, 19379 to 15162
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype                    
---  ------         --------------  -----                    
 0   url            19380 non-null  object                   
 1   host           19380 non-null  object                   
 2   title          19380 non-null  object                   
 3   text           19380 non-null  object                   
 4   date           19380 non-null  datetime64[ns, tzlocal()]
 5   patched_title  19380 non-null  object                   
 6   patched_text   19380 non-null  object                   
 7   category       19380 non-null  object                   
dtypes: datetime64[ns, tzlocal()](1), object(7)
memory usage: 1.

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

from purano.models import Document

def parse_db(file_name):
    db_engine = "sqlite:///{}".format(file_name)
    engine = create_engine(db_engine)
    Session = sessionmaker(bind=engine)
    session = Session()
    query = session.query(Document)
    docs = query.all()
    records = []
    for doc in docs:
        records.append({
            "title": doc.title,
            "text": doc.text,
            "url": doc.url,
            "host": doc.host,
            "timestamp": doc.date,
            "patched_title": doc.patched_title,
            "patched_text": doc.patched_text
        })
    return records

train_records = parse_db("0525_parsed.db")
test_0527_records = parse_db("0527_parsed.db")
test_0529_records = parse_db("0529_parsed.db")
print(len(train_records))
print(len(test_0527_records))
print(len(test_0529_records))
print(train_records[0])

19380
20080
19096
{'title': 'История разработки Saints Row', 'text': 'В апреле 2020 компания Deep Silver анонсировала майский релиз Saints Row: The Third Remastered, обновленную версию третьей части всем известного конкурента GTA, серии Saints Row. Игра вышла на всех современных платформах кроме Nintendo Switch и, возможно, призвана вернуть интерес игроков к линейке перед возможным скорым выходом долгожданной пятой части. В начале 2000-х  Rockstar Games  взорвала индустрию, за четыре года подарив ей целых три тайтла, которые перевернули, набирающий обороты, жанр игр  в открытом мире . Само собой, Rockstar была не единственной компанией, работающей с подобной концепцией, но именно GTA 3, GTA: Vice City, и GTA: San Andreas вывели песочницы с видом от третьего лица в мейнстрим. Другие компании, конечно же, не собирались долго сидеть в стороне и тут же принялись пополнять новоиспеченный жанр играми, большинство которых в скором времени получили ярлык «клона GTA». Одним из издателей, заинте

In [ ]:
# Fasttext model tuned on older documents
!wget https://www.dropbox.com/s/vttjivmmxw7leea/ru_vectors_v3.bin
# Clustering training markup
!wget https://www.dropbox.com/s/8lu6dw8zcrn840j/ru_clustering_0525_urls.tsv
# Clustering test set
!wget https://www.dropbox.com/s/v2lbfft6pgzoeq3/ru_clustering_test_fixed.tsv
# Headline selection training set
!wget https://www.dropbox.com/s/jpcwryaeszqtrf9/titles_markup_0525_urls.tsv
# Headline selection test set
!wget https://www.dropbox.com/s/v3c0q6msy2pwvfm/titles_markup_test.tsv

--2021-02-26 11:51:38--  https://www.dropbox.com/s/vttjivmmxw7leea/ru_vectors_v3.bin
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6022:18::a27d:4212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/vttjivmmxw7leea/ru_vectors_v3.bin [following]
--2021-02-26 11:51:39--  https://www.dropbox.com/s/raw/vttjivmmxw7leea/ru_vectors_v3.bin
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc59997fe24e7b633c659d64a247.dl.dropboxusercontent.com/cd/0/inline/BJp3o9FyKnmFqlD-ytV_CoWwFDQhHhNH1RsC0Ve76tkrePyKjybYF0hXzh_JFwjwXjcIEcKaxIw8shcUCI7bTZTG6RXHluQAXLL9q4FwK_9ZUcKmp6b8Wj0YeFGffn_qi_k/file# [following]
--2021-02-26 11:51:39--  https://uc59997fe24e7b633c659d64a247.dl.dropboxusercontent.com/cd/0/inline/BJp3o9FyKnmFqlD-ytV_CoWwFDQhHhNH1RsC0Ve76tkrePyKjybYF0hXzh_JFwjwXjcIEcKaxIw8shcUCI7bTZTG6RXHl

# Fetching texts for pretraining

Optional section, required for TF-IDF and Text-Title models

Full list of archives: https://github.com/IlyaGusev/purano/blob/master/raw_datasets.txt

In [ ]:
%%writefile raw_datasets.txt
https://data-static.usercontent.dev/DataClusteringSample0410.tar.gz
https://data-static.usercontent.dev/DataClusteringSample1117.tar.gz

Overwriting raw_datasets.txt


In [ ]:
%%writefile download_datasets.sh
#!/bin/bash

# Download datasets
rm -rf raw_datasets
mkdir raw_datasets
cd raw_datasets
wget -i ../raw_datasets.txt
for F in *.tar.gz; do
    tar -xzf "$F"
    rm -f "$F"
done
cd ../

Writing download_datasets.sh


In [ ]:
!bash download_datasets.sh

--2021-02-20 14:13:57--  https://data-static.usercontent.dev/DataClusteringSample0410.tar.gz
Resolving data-static.usercontent.dev (data-static.usercontent.dev)... 178.128.255.27
Connecting to data-static.usercontent.dev (data-static.usercontent.dev)|178.128.255.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 918103137 (876M) [application/octet-stream]
Saving to: ‘DataClusteringSample0410.tar.gz’

DataClusteringSampl 100%[===================>] 875.57M  19.2MB/s    in 49s     

2021-02-20 14:14:47 (17.8 MB/s) - ‘DataClusteringSample0410.tar.gz’ saved [918103137/918103137]

--2021-02-20 14:14:47--  https://data-static.usercontent.dev/DataClusteringSample1117.tar.gz
Reusing existing connection to data-static.usercontent.dev:443.
HTTP request sent, awaiting response... 200 OK
Length: 910299179 (868M) [application/octet-stream]
Saving to: ‘DataClusteringSample1117.tar.gz’

DataClusteringSampl 100%[===================>] 868.13M  20.8MB/s    in 43s     

2021-02-2

In [ ]:
from purano.io import read_tg_html_dir
from tqdm.notebook import tqdm

from purano.run_parse import DocumentsCleaner

cleaner = DocumentsCleaner("configs/cleaner.jsonnet")

raw_texts = []
pretrain_records = []
for i, record in tqdm(enumerate(read_tg_html_dir("raw_datasets"))):
    cleaned_document = cleaner(record)
    if not cleaned_document:
        continue
    raw_texts.append(cleaned_document["title"])
    raw_texts.append(cleaned_document["text"])
    pretrain_records.append(cleaned_document)

Parsed 10000 documents
Parsed 20000 documents
Parsed 30000 documents
Parsed 40000 documents
Parsed 50000 documents
Parsed 60000 documents
Parsed 70000 documents
Parsed 80000 documents
Parsed 90000 documents
Parsed 100000 documents
Parsed 110000 documents
Parsed 120000 documents
Parsed 130000 documents
Parsed 140000 documents
Parsed 150000 documents
Parsed 160000 documents
Parsed 170000 documents
Parsed 180000 documents
Parsed 190000 documents
Parsed 200000 documents
Parsed 210000 documents
Parsed 220000 documents
Parsed 230000 documents
Parsed 240000 documents
Parsed 250000 documents
Parsed 260000 documents
Parsed 270000 documents
Parsed 280000 documents
Parsed 290000 documents
Parsed 300000 documents
Parsed 310000 documents
Parsed 320000 documents
Parsed 330000 documents
Parsed 340000 documents
Parsed 350000 documents
Parsed 360000 documents
Parsed 370000 documents
Parsed 380000 documents
Parsed 390000 documents
Parsed 400000 documents
Parsed 410000 documents
Parsed 420000 documents
P

In [ ]:
print(len(pretrain_records))

229601


# Clustering

## Evaluation

In [ ]:
from purano.io import read_markup_tsv

train_markup = read_markup_tsv("ru_clustering_0525_urls.tsv")
test_markup = read_markup_tsv("ru_clustering_test_fixed.tsv")

In [ ]:
from collections import Counter
from statistics import median, mean
from sklearn.cluster import AgglomerativeClustering
from purano.clusterer.metrics import calc_metrics

def get_quality(markup, embeds, records, dist_threshold, print_result=False):
    clustering_model = AgglomerativeClustering(
        n_clusters=None,
        distance_threshold=dist_threshold,
        linkage="average",
        affinity="cosine"
    )

    clustering_model.fit(embeds)
    labels = clustering_model.labels_

    idx2url = dict()
    url2record = dict()
    for i, record in enumerate(records):
        idx2url[i] = record["url"]
        url2record[record["url"]] = record

    url2label = dict()
    for i, label in enumerate(labels):
        url2label[idx2url[i]] = label
        
    metrics = calc_metrics(markup, url2record, url2label)[0]
    if print_result:
        print()
        print("Accuracy: {:.1f}".format(metrics["accuracy"] * 100.0))
        print("Positives Recall: {:.1f}".format(metrics["1"]["recall"] * 100.0))
        print("Positives Precision: {:.1f}".format(metrics["1"]["precision"] * 100.0))
        print("Positives F1: {:.1f}".format(metrics["1"]["f1-score"] * 100.0))
        print("Distance: ", dist_threshold)
        sizes = list(Counter(labels).values())
        print("Max cluster size: ", max(sizes))
        print("Median cluster size: ", median(sizes))
        print("Avg cluster size: {:.2f}".format(mean(sizes)))
        return
    return metrics["1"]["f1-score"]

In [ ]:
import copy

def form_submission(markup, embeds, records, dist_threshold, dataset_name):
    clustering_model = AgglomerativeClustering(
        n_clusters=None,
        distance_threshold=dist_threshold,
        linkage="average",
        affinity="cosine"
    )

    clustering_model.fit(embeds)
    labels = clustering_model.labels_
    idx2url = dict()
    url2record = dict()
    for i, record in enumerate(records):
        idx2url[i] = record["url"]
        url2record[record["url"]] = record
    url2label = dict()
    for i, label in enumerate(labels):
        url2label[idx2url[i]] = label
    submission = []
    for record in markup:
        dataset = record["dataset"]
        if dataset != dataset_name:
            continue
        first_url = record["first_url"]
        second_url = record["second_url"]
        r = copy.copy(record)
        r["quality"] = "OK" if url2label[first_url] == url2label[second_url] else "BAD"
        submission.append(r)
    return submission

## FastText model

In [ ]:
import fasttext
import numpy as np
from tqdm import tqdm

def ft_embed_text(text, ft_model, max_tokens_count=150):
    vector_dim = ft_model.get_dimension()
    tokens = text.split(" ")[:max_tokens_count]
    word_embeddings = np.zeros((len(tokens), vector_dim), dtype=np.float32)
    for token_num, token in enumerate(tokens):
        word_embeddings[token_num, :] = ft_model.get_word_vector(token)
    mean_embedding = np.mean(word_embeddings, axis=0)
    max_embeddings = np.max(word_embeddings, axis=0)
    return np.concatenate((mean_embedding, max_embeddings), axis=0)

def ft_records_to_embeds(records, ft_model):
    ft_embeddings = np.zeros((len(records), ft_model.get_dimension() * 2), dtype=np.float32)
    for i, record in enumerate(tqdm(records)):
        sample = record["patched_title"] + " " + record["patched_text"]
        ft_embeddings[i, :] = ft_embed_text(sample, ft_model)
    return ft_embeddings

ft_model = fasttext.load_model("ru_vectors_v3.bin")
train_ft_embeddings = ft_records_to_embeds(train_records, ft_model)
test_0527_ft_embeddings = ft_records_to_embeds(test_0527_records, ft_model)
test_0529_ft_embeddings = ft_records_to_embeds(test_0529_records, ft_model)

100%|██████████| 19096/19096 [00:30<00:00, 626.62it/s]


In [ ]:
get_quality(train_markup, train_ft_embeddings, train_records, 0.041, print_result=True)


Accuracy: 80.9
Positives Recall: 79.4
Positives Precision: 81.4
Positives F1: 80.4
Distance:  0.041
Max cluster size:  320
Median cluster size:  2.0
Avg cluster size: 6.29


In [ ]:
from purano.io import write_markup_tsv
submission_0527_ft = form_submission(test_markup, test_0527_ft_embeddings, test_0527_records, 0.041, "0527")
submission_0529_ft = form_submission(test_markup, test_0529_ft_embeddings, test_0529_records, 0.041, "0529")
full_submission_ft = submission_0527_ft + submission_0529_ft
write_markup_tsv(full_submission_ft, "answer.txt", res_key="quality")
!rm -f submission_ft.zip
!zip submission_ft.zip answer.txt

  adding: answer.txt (deflated 68%)


In [ ]:
!head answer.txt

quality	dataset	second_url	first_url
OK	0527	https://saratov.mk.ru/social/2020/05/27/v-saratovskoy-oblasti-na-vremya-zakonchilas-chereda-dozhdlivykh-dney.html	https://tverigrad.ru/publication/posle-vyhodnyh-v-tverskoj-oblasti-rezko-poholodaet
BAD	0527	https://top68.ru/news/114229-zernouborochnye-kombayny-v-tambovskoy-oblasti-osnastyat-sistemoy-bespilotnogo-vozhdeniya	https://lenta.ru/news/2020/05/27/beton/
BAD	0527	http://ufacitynews.ru/news/2020/05/27/v-ufe-muzhchina-lishilsya-nedvizhimosti-stoimostyu-bolee-20-mln-rublej-iz-za-dolgov/	https://bk55.ru/news/article/168850/
BAD	0527	https://www.obozrevatel.com/kiyany/crime/v-kieve-prohozhij-pogib-v-yame-vyiryitoj-remontnikami.htm	https://inforesist.org/v-kieve-obnaruzhili-telo-muzhchiny-v-kotlovane-s-vodoj-kotoryj-vyryli-kommunalshhiki/
BAD	0527	https://android-robot.com/v-bolnice-temperaturu-vrachej-na-vxode-proveryaet-robot/	https://realnoevremya.ru/news/175925-drkb-poluchila-200-zaschitnyh-schitkov-dlya-pacientov-s-covid-19
BAD	0527	h

## USE model

In [ ]:
!pip install tensorflow-text tensorflow-hub

     |████████████████████████████████| 3.4MB 11.4MB/s 


In [ ]:
import tensorflow as tf
import tensorflow_text
import tensorflow_hub as hub

module_url = "https://tfhub.dev/google/universal-sentence-encoder-multilingual/3"
use_model = hub.load(module_url)
print("module %s loaded" % module_url)

module https://tfhub.dev/google/universal-sentence-encoder-multilingual/3 loaded


In [ ]:
import numpy as np
import tqdm.notebook as tq


def use_get_embedding(text, model):
    return model([text]).numpy()[0]

def use_records_to_embeds(records, model):
    embeddings = np.zeros((len(records), 512))
    for i, record in tq.tqdm(enumerate(records)):
        embeddings[i] = use_get_embedding(record["title"] + " " + record["text"], model)
    return embeddings


train_use_embeddings = use_records_to_embeds(train_records, use_model)
test_0527_use_embeddings = use_records_to_embeds(test_0527_records, use_model)
test_0529_use_embeddings = use_records_to_embeds(test_0529_records, use_model)

In [ ]:
get_quality(train_markup, train_use_embeddings, train_records, 0.37, print_result=True)


Accuracy: 89.4
Positives Recall: 89.7
Positives Precision: 89.0
Positives F1: 89.3
Distance:  0.37
Max cluster size:  187
Median cluster size:  1.0
Avg cluster size: 2.85


In [ ]:
from purano.io import write_markup_tsv
submission_0527_use = form_submission(test_markup, test_0527_use_embeddings, test_0527_records, 0.37, "0527")
submission_0529_use = form_submission(test_markup, test_0529_use_embeddings, test_0529_records, 0.37, "0529")
full_submission_use = submission_0527_use + submission_0529_use
write_markup_tsv(full_submission_use, "answer.txt", res_key="quality")
!rm -f submission_use.zip
!zip submission_use.zip answer.txt

  adding: answer.txt (deflated 68%)


## TF-IDF model

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from purano.util import tokenize_to_lemmas

tfidf_vectorizer = TfidfVectorizer(tokenizer=tokenize_to_lemmas, max_df=0.3, min_df=5)
old_embeddings = tfidf_vectorizer.fit_transform(raw_texts[-150000:])
svd = TruncatedSVD(n_components=500, n_iter=20, random_state=42)
svd.fit(old_embeddings)

TruncatedSVD(algorithm='randomized', n_components=500, n_iter=20,
             random_state=42, tol=0.0)

In [ ]:
import numpy as np

def tfidf_records_to_embeds(records, vectorizer, svd):
    embeddings = vectorizer.transform([r["title"] + " " + r["text"] for r in records])
    embeddings = svd.transform(embeddings)
    embeddings[np.logical_and(np.max(embeddings, axis=1) == 0, np.sum(embeddings, axis=1) == 0)] = np.random.rand(embeddings.shape[1])
    return embeddings

train_tfidf_embeddings = tfidf_records_to_embeds(train_records, tfidf_vectorizer, svd)
test_0527_tfidf_embeddings = tfidf_records_to_embeds(test_0527_records, tfidf_vectorizer, svd)
test_0529_tfidf_embeddings = tfidf_records_to_embeds(test_0529_records, tfidf_vectorizer, svd)

In [ ]:
get_quality(train_markup, train_tfidf_embeddings, train_records, 0.5, print_result=True)


Accuracy: 83.4
Positives Recall: 83.1
Positives Precision: 83.2
Positives F1: 83.1
Distance:  0.5
Max cluster size:  166
Median cluster size:  2.0
Avg cluster size: 3.83


In [ ]:
from purano.io import write_markup_tsv
submission_0527_tfidf = form_submission(test_markup, test_0527_tfidf_embeddings, test_0527_records, 0.5, "0527")
submission_0529_tfidf = form_submission(test_markup, test_0529_tfidf_embeddings, test_0529_records, 0.5, "0529")
full_submission_tfidf = submission_0527_tfidf + submission_0529_tfidf
write_markup_tsv(full_submission_tfidf, "answer.txt", res_key="quality")
!rm -f submission_tfidf.zip
!zip submission_tfidf.zip answer.txt

  adding: answer.txt (deflated 68%)


## Text-Title siamise model over FastText

In [ ]:
import numpy as np
from hnswlib import Index as HnswIndex

from purano.util import tokenize

def ft_words_to_embeddings(model, words):
    vector_dim = model.get_dimension()
    embeddings = np.zeros((len(words), vector_dim))
    for i, w in enumerate(words):
        embeddings[i] = model.get_word_vector(w)
        embeddings[i] /= np.linalg.norm(embeddings[i])
    return embeddings

def ft_embed_text(model, text):
    words = tokenize(text)
    norm_vectors = ft_words_to_embeddings(model, words)
    avg_wv = np.mean(norm_vectors, axis=0)
    max_wv = np.max(norm_vectors, axis=0)
    min_wv = np.min(norm_vectors, axis=0)
    return np.concatenate((avg_wv, max_wv, min_wv))

# Index for mining hard negatives
class FasttextHnsw:
    def __init__(self, model):
        self.model = model
        self.vector_dim = model.get_dimension()
        self.hnsw = HnswIndex(space='l2', dim=self.vector_dim * 3)

    def build_hnsw(self, texts):
        n = len(texts)
        self.hnsw.init_index(max_elements=n, ef_construction=100, M=16)
        embeddings = np.zeros((n, self.vector_dim * 3))
        for i, text in enumerate(texts):
            embeddings[i] = ft_embed_text(self.model, text)
        self.hnsw.add_items(embeddings)

In [ ]:
import random

import numpy as np
import torch
import fasttext
from torch.utils.data import Dataset
from tqdm.notebook import tqdm


from purano.util import tokenize
from purano.training.fasttext_hnsw import FasttextHnsw


class Text2TitleDataset(Dataset):
    def __init__(self, data, ft_model, min_words=2, max_words=150):
        self.ft_hnsw = FasttextHnsw(ft_model)
        self.ft_hnsw.build_hnsw([r["title"] for r in data])

        self.samples = []
        for row in tqdm(data):
            title_words = tokenize(row["title"])
            text_words = tokenize(row["text"])[:max_words]
            if len(text_words) < min_words or len(title_words) < min_words:
                continue
            title = " ".join(title_words)
            text = " ".join(text_words)

            text_vector = ft_embed_text(ft_model, text)
            title_vector = ft_embed_text(ft_model, title)

            labels = list(self.ft_hnsw.hnsw.knn_query(title_vector, k=20)[0][0])[1:]
            random.shuffle(labels)
            bad_indices = labels[:2] + [random.randint(0, len(data) - 1)]
            bad_vectors = self.ft_hnsw.hnsw.get_items(bad_indices)
            for bad_vector in bad_vectors:
                bad_vector = np.array(bad_vector)
                assert bad_vector.shape == title_vector.shape == text_vector.shape
                assert bad_vector.dtype == title_vector.dtype == text_vector.dtype
                sample = (
                    torch.FloatTensor(text_vector),
                    torch.FloatTensor(title_vector),
                    torch.FloatTensor(bad_vector)
                )
                self.samples.append(sample)

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, index):
        return self.samples[index]

In [ ]:
import torch
import torch.nn as nn
from pytorch_lightning import LightningModule

class Embedder(nn.Module):
    def __init__(self, embedding_dim, hidden_dim):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.mapping_layer = nn.Linear(embedding_dim, hidden_dim)

    def forward(self, in_vectors):
        projections = self.mapping_layer(in_vectors)
        norm = projections.norm(p=2, dim=1, keepdim=True)
        projections = projections.div(norm)
        return projections


class Text2TitleModel(LightningModule):
    def __init__(self, embedding_dim=384, hidden_dim=128):
        super().__init__()

        self.text_embedder = Embedder(embedding_dim, hidden_dim)
        self.title_embedder = Embedder(embedding_dim, hidden_dim)
        self.distance = nn.PairwiseDistance(p=2)
        self.margin = 0.3

    def forward(self, pivot_vectors, positive_vectors, negative_vectors):
        pivot = self.text_embedder(pivot_vectors)
        positive = self.title_embedder(positive_vectors)
        negative = self.title_embedder(negative_vectors)
        distances = self.distance(pivot, positive) - self.distance(pivot, negative) + self.margin
        loss = torch.mean(torch.max(distances, torch.zeros_like(distances)))
        return loss

    def training_step(self, batch, batch_nb):
        train_loss = self(*batch)
        self.log("train_loss", train_loss, prog_bar=True, logger=True)
        return train_loss

    def validation_step(self, batch, batch_nb):
        val_loss = self(*batch)
        self.log("val_loss", val_loss, prog_bar=True, logger=True)

    def test_step(self, batch, batch_nb):
        test_loss = self(*batch)
        self.log("test_loss", test_loss, prog_bar=True, logger=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return [optimizer]

In [ ]:
import random
from torch.utils.data import DataLoader, RandomSampler
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping

BATCH_SIZE = 64
NUM_WORKERS = 5
MAX_WORDS = 150
EPOCHS = 100
PATIENCE = 4

pretrain_records.sort(key=lambda x: x.get("date"))
tt_records = pretrain_records[:150000]
val_border = int(0.9 * len(tt_records))
tt_train_records = tt_records[:val_border]
tt_val_records = tt_records[val_border:]

ft_model = fasttext.load_model("ru_vectors_v3.bin")
train_data = Text2TitleDataset(tt_train_records, ft_model, max_words=MAX_WORDS)
train_sampler = RandomSampler(train_data)
train_loader = DataLoader(train_data,
    sampler=train_sampler,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS
)

val_data = Text2TitleDataset(tt_val_records, ft_model, max_words=MAX_WORDS)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)

tt_model = Text2TitleModel()
early_stop_callback = EarlyStopping(
    monitor="val_loss",
    min_delta=0.0,
    patience=PATIENCE,
    verbose=True,
    mode="min"
)
    
trainer = Trainer(
    gpus=0,
    checkpoint_callback=False,
    accumulate_grad_batches=1,
    max_epochs=EPOCHS,
    callbacks=[early_stop_callback],
    val_check_interval=1.0,
    progress_bar_refresh_rate=100,
    deterministic=True
)
trainer.fit(tt_model, train_loader, val_loader)

GPU available: True, used: False
TPU available: None, using: 0 TPU cores


/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: GPU available but not used. Set the --gpus flag when calling the script.
  warnings.warn(*args, **kwargs)

  | Name           | Type             | Params
----------------------------------------------------
0 | text_embedder  | Embedder         | 49.3 K
1 | title_embedder | Embedder         | 49.3 K
2 | distance       | PairwiseDistance | 0     
----------------------------------------------------
98.6 K    Trainable params
0         Non-trainable params
98.6 K    Total params
0.394     Total estimated model params size (MB)


1

In [ ]:
from tqdm.notebook import tqdm

def tt_records_to_embeds(records, ft_model, tt_model, max_words=150):
    title_embedder = tt_model.title_embedder
    text_embedder = tt_model.text_embedder
    embeddings = np.zeros((len(records), text_embedder.hidden_dim + title_embedder.hidden_dim))
    for i, record in tqdm(enumerate(records)):
        title_words = tokenize(record["title"])
        if not title_words:
            title_words = ["нет"]
        text_words = tokenize(record["text"])[:max_words]
        if not text_words:
            text_words = ["нет"]
        title_embedding = ft_embed_text(ft_model, " ".join(title_words))
        text_embedding = ft_embed_text(ft_model, " ".join(text_words))
        title_embedding = title_embedder(torch.FloatTensor(title_embedding).unsqueeze(0)).squeeze(0).detach().cpu().numpy()
        text_embedding = text_embedder(torch.FloatTensor(text_embedding).unsqueeze(0)).squeeze(0).detach().cpu().numpy()
        embedding = np.concatenate((title_embedding, text_embedding), axis=0)
        embeddings[i] = embedding
    return embeddings

train_tt_embeddings = tt_records_to_embeds(train_records, ft_model, tt_model, MAX_WORDS)
test_0527_tt_embeddings = tt_records_to_embeds(test_0527_records, ft_model, tt_model)
test_0529_tt_embeddings = tt_records_to_embeds(test_0529_records, ft_model, tt_model)

In [ ]:
get_quality(train_markup, train_tt_embeddings, train_records, 0.38, print_result=True)


Accuracy: 87.2
Positives Recall: 88.0
Positives Precision: 86.3
Positives F1: 87.1
Distance:  0.38
Max cluster size:  143
Median cluster size:  2.0
Avg cluster size: 4.07


In [ ]:
from purano.io import write_markup_tsv
submission_0527_tt = form_submission(test_markup, test_0527_tt_embeddings, test_0527_records, 0.38, "0527")
submission_0529_tt = form_submission(test_markup, test_0529_tt_embeddings, test_0529_records, 0.38, "0529")
full_submission_tt = submission_0527_tt + submission_0529_tt
write_markup_tsv(full_submission_tt, "answer.txt", res_key="quality")
!rm -f submission_tt.zip
!zip submission_tt.zip answer.txt

  adding: answer.txt (deflated 68%)


# Headline selection

In [ ]:
from purano.io import read_markup_tsv

hl_train_markup = read_markup_tsv("titles_markup_0525_urls.tsv")
hl_test_markup = read_markup_tsv("titles_markup_test.tsv")

## USE + Catboost

In [ ]:
import tensorflow as tf
import tensorflow_text
import tensorflow_hub as hub
import numpy as np
import tqdm.notebook as tq

module_url = "https://tfhub.dev/google/universal-sentence-encoder-multilingual/3"
use_model = hub.load(module_url)
print("module %s loaded" % module_url)

def use_get_embedding(text, model):
    return model([text]).numpy()[0]

def use_records_to_embeds(records, model):
    embeddings = np.zeros((len(records), 512))
    for i, record in tq.tqdm(enumerate(records)):
        embeddings[i] = use_get_embedding(record["title"], model)
    return embeddings

train_use_title_embeddings = use_records_to_embeds(train_records, use_model)
test_0527_use_embeddings = use_records_to_embeds(test_0527_records, use_model)
test_0529_use_embeddings = use_records_to_embeds(test_0529_records, use_model)

module https://tfhub.dev/google/universal-sentence-encoder-multilingual/3 loaded


In [ ]:
import random
from catboost import Pool
from collections import Counter

def get_groups(markup):
    groups = list()
    url2group = dict()
    for r in markup:
        if r["quality"] == "draw" or r["quality"] == "bad":
            continue
        left_url = r["left_url"]
        right_url = r["right_url"]
        left_group_id = url2group.get(left_url, None)
        right_group_id = url2group.get(right_url, None)
        if left_group_id is not None and right_group_id is None:
            groups[left_group_id].add(right_url)
            url2group[right_url] = left_group_id
        elif right_group_id is not None and left_group_id is None:
            groups[right_group_id].add(left_url)
            url2group[left_url] = right_group_id
        elif left_group_id is None and right_group_id is None:
            groups.append({left_url, right_url})
            url2group[left_url] = url2group[right_url] = len(groups) - 1
        elif left_group_id != right_group_id:
            for u in groups[right_group_id]:
                url2group[u] = left_group_id
            groups[left_group_id] = groups[left_group_id] | groups[right_group_id]
            groups[right_group_id] = set()
        assert right_url in groups[url2group[right_url]]
        assert left_url in groups[url2group[left_url]]
        assert url2group[right_url] == url2group[left_url]
    groups = [group for group in groups if group]
    url2group = dict()
    for i, group in enumerate(groups):
        for url in group:
            url2group[url] = i
    return groups, url2group


def get_features(markup, url2record, embeddings, url2id):
    urls = set()
    for r in markup:
        if r["quality"] == "draw" or r["quality"] == "bad":
            continue
        urls.add(r["left_url"])
        urls.add(r["right_url"])
    features = dict()
    for url in urls:
        record = url2record[url]
        embedding = list(embeddings[url2id[url]])
        features[url] = embedding
    return features


def get_pairs(markup, url2group, is_train_group):
    all_pairs = list()
    train_pairs = list()
    val_pairs = list()
    for r in markup:
        if r["quality"] == "draw" or r["quality"] == "bad":
            continue
        left_url = r["left_url"]
        right_url = r["right_url"]
        group_id = url2group[left_url]
        assert url2group[right_url] == group_id
        is_train = is_train_group[group_id]
        pairs = train_pairs if is_train else val_pairs
        if r["quality"] == "left":
            pairs.append((left_url, right_url))
            all_pairs.append((left_url, right_url))
        elif r["quality"] == "right":
            pairs.append((right_url, left_url))
            all_pairs.append((right_url, left_url))
    return all_pairs, train_pairs, val_pairs


def convert_to_cb_pool(groups, pairs, features):
    urls_set = set()
    for url1, url2 in pairs:
        urls_set.add(url1)
        urls_set.add(url2)
    urls = []
    group_id = []
    for i, group in enumerate(groups):
        for url in group:
            if url not in urls_set:
                continue
            group_id.append(i)
            urls.append(url)
    urls2id = {url: i for i, url in enumerate(urls)}
    features = [features[url] for url in urls]
    pairs = [(urls2id[url1], urls2id[url2]) for url1, url2 in pairs]
    pool = Pool(data=features, pairs=pairs, group_id=group_id)
    return pool, urls


train_url2record = {r["url"]: r for r in train_records}
train_url2id = {r["url"]: i for i, r in enumerate(train_records)}

val_part = 0.1
train_part = 1.0 - val_part
groups, url2group = get_groups(hl_train_markup)
is_train_group = [random.random() < train_part for _ in range(len(groups))]

features = get_features(hl_train_markup, train_url2record, train_use_title_embeddings, train_url2id)
all_pairs, train_pairs, val_pairs = get_pairs(hl_train_markup, url2group, is_train_group)

train_pool, train_urls = convert_to_cb_pool(groups, train_pairs, features)
val_pool, val_urls = convert_to_cb_pool(groups, val_pairs, features)

In [ ]:
from catboost import CatBoost

cb_model = CatBoost(params={"loss_function": "PairLogit", "task_type": "GPU"})

cb_model.fit(train_pool, eval_set=val_pool)

0:	learn: 0.6812022	test: 0.6788095	best: 0.6788095 (0)	total: 53.2ms	remaining: 53.2s
1:	learn: 0.6695208	test: 0.6695656	best: 0.6695656 (1)	total: 95.2ms	remaining: 47.5s
2:	learn: 0.6581827	test: 0.6578059	best: 0.6578059 (2)	total: 137ms	remaining: 45.5s
3:	learn: 0.6471893	test: 0.6475940	best: 0.6475940 (3)	total: 179ms	remaining: 44.5s
4:	learn: 0.6387101	test: 0.6401899	best: 0.6401899 (4)	total: 231ms	remaining: 45.9s
5:	learn: 0.6304563	test: 0.6318356	best: 0.6318356 (5)	total: 276ms	remaining: 45.8s
6:	learn: 0.6213128	test: 0.6243118	best: 0.6243118 (6)	total: 317ms	remaining: 45s
7:	learn: 0.6125291	test: 0.6162686	best: 0.6162686 (7)	total: 364ms	remaining: 45.2s
8:	learn: 0.6052745	test: 0.6088652	best: 0.6088652 (8)	total: 417ms	remaining: 45.9s
9:	learn: 0.5964792	test: 0.5991916	best: 0.5991916 (9)	total: 457ms	remaining: 45.2s
10:	learn: 0.5910875	test: 0.5963251	best: 0.5963251 (10)	total: 507ms	remaining: 45.6s
11:	learn: 0.5845180	test: 0.5911623	best: 0.5911623

In [ ]:
val_predictions = cb_model.predict(val_pool)
val_urls2id = {url: i for i, url in enumerate(val_urls)}
correct_pairs = 0
all_pairs = 0
for winner_url, loser_url in val_pairs:
    all_pairs += 1
    winner_id = val_urls2id[winner_url]
    loser_id = val_urls2id[loser_url]
    winner_pred = val_predictions[winner_id]
    loser_pred = val_predictions[loser_id]
    correct_pairs += 1 if winner_pred > loser_pred else 0
print(float(correct_pairs) / all_pairs)

0.8116760828625236


In [ ]:
import copy

def hl_form_submission(markup, records, embeddings, cb_model, dataset_name, draw_border=0.1):
    predictions = cb_model.predict(embeddings)

    url2idx = dict()
    for i, record in enumerate(records):
        url = record["url"]
        url2idx[url] = i

    submission = []
    for record in markup:
        if record["dataset"] != dataset_name:
            continue

        left_url = record["left_url"]
        right_url = record["right_url"]
        left_idx = url2idx[left_url]
        right_idx = url2idx[right_url]
      
        left_pred = predictions[left_idx]
        right_pred = predictions[right_idx]
        result = "left" if left_pred > right_pred else "right"
        result = "draw" if abs(left_pred - right_pred) < draw_border else result

        r = copy.copy(record)
        r["quality"] = result
        submission.append(r)

    return submission

In [ ]:
from purano.io import write_markup_tsv
hl_use_cb_submission_0527 = hl_form_submission(hl_test_markup, test_0527_records, test_0527_use_embeddings, cb_model, "0527")
hl_use_cb_submission_0529 = hl_form_submission(hl_test_markup, test_0529_records, test_0529_use_embeddings, cb_model, "0529")
hl_use_cb_full_submission = hl_use_cb_submission_0527 + hl_use_cb_submission_0529
write_markup_tsv(hl_use_cb_full_submission, "answer.txt", res_key="quality")
!rm -f hl_use_cb_submission.zip
!zip hl_use_cb_submission.zip answer.txt

  adding: answer.txt (deflated 82%)


# Headline generation

In [4]:
!wget https://www.dropbox.com/s/opgtoc8ucjhhyii/hg_texts.jsonl.tar.gz
!tar -xzvf hg_texts.jsonl.tar.gz

--2021-04-23 17:15:51--  https://www.dropbox.com/s/opgtoc8ucjhhyii/hg_texts.jsonl.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/opgtoc8ucjhhyii/hg_texts.jsonl.tar.gz [following]
--2021-04-23 17:15:51--  https://www.dropbox.com/s/raw/opgtoc8ucjhhyii/hg_texts.jsonl.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc429a4a11faf5463efc2d1beb3b.dl.dropboxusercontent.com/cd/0/inline/BNK3F0_hNpuR6G3YydaViiMLwQrYgoHslunorxYunLDUFHMwv7kLkuyMF8GgdNQ97mTxuox0cSOLqjoHFr7HuFkmTySEYZgTecQI-Xnf_RuMfRaqYAR6uANXDypEGjC73c6Wm_aW0xpgkcthmnywPChs/file# [following]
--2021-04-23 17:15:51--  https://uc429a4a11faf5463efc2d1beb3b.dl.dropboxusercontent.com/cd/0/inline/BNK3F0_hNpuR6G3YydaViiMLwQrYgoHslunorxYunLDUFHMwv7kLkuyMF8GgdNQ

In [5]:
import json

hg_records = []
with open("hg_texts.jsonl", "r") as r:
    for line in r:
        hg_records.append(json.loads(line))
print(hg_records[0])

{'text': 'Решение Facebook заблокировать ряд публикаций российских СМИ неприемлемо, оно нарушает национальное законодательство России, а также право на распространение и получение информации, заявил председатель Госдумы Вячеслав Володин. «Решение компании Facebook нарушает основополагающие права и свободы граждан, право на распространение и получение информации», — сказал Володин. Он напомнил, что со стороны Facebook подобное происходит не впервые, а ранее так же поступали Twitter и Instagram. «Мы будем предлагать законодательные решения, которые не допускали бы подобного в отношении российских СМИ», — приводит его слова пресс-служба Госдумы. Володин подчеркнул, что эта тема станет приоритетной в повестке ближайших международных встреч, в рамках которых российская сторона обменяется предложениями по решению этой проблемы с коллегами из других стран. Как ранее стало известно, материал RT о задержании в Воронеже украинских радикалов оказался заблокирован администрацией социальной сети Fa

## Lead-1

In [ ]:
import razdel
import random
from collections import defaultdict

id2hyps = defaultdict(list)
with open("hg_texts.jsonl", "r") as r:
    for line in r:
        record = json.loads(line)
        sample_id = record["sample_id"]
        text = record["text"]
        first_sentence = list(razdel.sentenize(text))[0].text.replace(".", "")
        id2hyps[sample_id].append(first_sentence)

with open("answer.txt", "w") as w:
    for sample_id, hyps in id2hyps.items():
        hyp = random.choice(hyps)
        w.write(json.dumps({
            "sample_id": sample_id,
            "title": hyp
        }, ensure_ascii=False) + "\n")

!zip submission_lead1.zip answer.txt

  adding: answer.txt (deflated 72%)


## Trained EncoderDecoder RuBERT

In [6]:
from transformers import AutoTokenizer, EncoderDecoderModel

model_name = "IlyaGusev/rubert_telegram_headlines"
tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=False, do_basic_tokenize=False, strip_accents=False)
model = EncoderDecoderModel.from_pretrained(model_name).cuda()

def predict_headline(text):
    input_ids = tokenizer(
        [text],
        add_special_tokens=True,
        max_length=256,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )["input_ids"]

    output_ids = model.generate(
        input_ids=input_ids.cuda(),
        max_length=64,
        no_repeat_ngram_size=3,
        num_beams=6,
        top_p=0.95
    )[0]

    headline = tokenizer.decode(output_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    return headline

In [7]:
predict_headline(hg_records[0]["text"])

'Володин раскритиковал Facebook за блокировку публикаций в соцсетях'

In [10]:
import random
from collections import defaultdict
from tqdm.notebook import tqdm

id2hyps = defaultdict(list)
random.seed(13332417)

for record in tqdm(hg_records):
    text = record["text"]
    sample_id = record["sample_id"]
    text_id = record["text_id"]
    prediction = predict_headline(text)
    id2hyps[sample_id].append(prediction)


with open("answer.txt", "w") as w:
    for sample_id, hyps in id2hyps.items():
        hyp = random.choice(hyps)
        w.write(json.dumps({
            "title": hyp,
            "sample_id": sample_id
        }, ensure_ascii=False).strip() + "\n")

!zip submission_hg_rubert.zip answer.txt


  adding: answer.txt (deflated 73%)
